In [1]:
from collections import deque

In [2]:
conn = {}
start = None

filepath = "./data/day10.txt"

with open(filepath) as f:
    row = 0
    while line := f.readline():
        line = line.rstrip()
        for col, c in enumerate(line):
            pos = (row, col)
            if c == 'S':
                start = pos
            elif c == '.':
                continue
            elif c == 'F':
                conn[pos] = [(row+1, col), (row, col+1)]
            elif c == '7':
                conn[pos] = [(row+1, col), (row, col-1)]
            elif c == 'J':
                conn[pos] = [(row-1, col), (row, col-1)]
            elif c == 'L':
                conn[pos] = [(row-1, col), (row, col+1)]
            elif c == '-':
                conn[pos] = [(row, col-1), (row, col+1)]
            elif c == '|':
                conn[pos] = [(row+1, col), (row-1, col)]
            else:
                print('we should not be here!', c)
        row += 1

In [3]:
start

(83, 25)

In [4]:
conn[start] = [(start[0]+i, start[1]) for i in [-1, 1]] + [(start[0], start[1]+i) for i in [-1, 1]]
conn[start]

[(82, 25), (84, 25), (83, 24), (83, 26)]

In [5]:
q = deque()
q.append(start + (0,))
visited = {start}
m = 0
while q:
    row, col, d = q.pop()
    m = d
    for row2, col2 in conn[(row, col)]:
        if (row2, col2) in visited:
            continue
        if (row2, col2) not in conn:
            continue
        if (row, col) not in conn[(row2, col2)]:
            continue
        q.appendleft((row2, col2, d+1))
        visited.add((row2, col2))
m

6754

In [6]:
loop = {start}
q = deque()
q.append(start)
while q:
    node = q.pop()
    for adj_node in conn[node]:
        if adj_node in loop:
            continue
        if adj_node not in conn:
            continue
        if node not in conn[adj_node]:
            continue
        q.appendleft(adj_node)
        loop.add(adj_node)

In [7]:
len(loop)

13508

In [8]:
max_row, max_col = 140, 140

In [9]:
def connected(node1, node2):
    if (node1 not in conn) or (node2 not in conn):
        return False
    return (node1 in conn[node2]) and (node2 in conn[node1])

In [10]:
big_loop = set()
for row, col in loop:
    big_loop.add((2*row, 2*col))
    for i in [-1, 1]:
        if connected((row, col), (row+i, col)):
            big_loop.add((2*row+i, 2*col))
        if connected((row, col), (row, col+i)):
            big_loop.add((2*row, 2*col+i))

In [11]:
len(big_loop), 2*len(loop)

(27016, 27016)

In [12]:
outside_points = set()
for r in range(max_row):
    for c in range(max_col):
        point = (r, c)
        if point in loop:
            continue
        if point in outside_points:
            continue
        outside_flag = False
        path = set()
        path.add((2*r, 2*c))
        q = deque()
        q.append((2*r, 2*c))
        while q and not outside_flag:
            row, col = q.pop()
            for i in [-1, 1]:
                if row+i < 0 or row+i >= 2*max_row:
                    outside_flag = True
                    continue
                if (row+i, col) in big_loop:
                    continue
                if (row+i, col) in path:
                    continue
                path.add((row+i, col))
                q.appendleft((row+i, col))
            for i in [-1, 1]:
                if col+i < 0 or col+i >= 2*max_col:
                    outside_flag = True
                    continue
                if (row, col+i) in big_loop:
                    continue
                if (row, col+i) in path:
                    continue
                path.add((row, col+i))
                q.appendleft((row, col+i))
        if outside_flag:
            for a, b in path:
                if a%2 == 0 and b%2 == 0:
                    outside_points.add((a//2, b//2))

In [13]:
max_row*max_col, len(loop), len(outside_points), max_row*max_col-len(loop)-len(outside_points)

(19600, 13508, 5525, 567)